In [1]:
#Dependencies
import os
import pandas as pd

In [2]:
#read the csv files and load the DataFrames
schools="schools_complete.csv"
students="students_complete.csv"
school_data=pd.read_csv(schools,encoding="iso-8859-1")
student_data=pd.read_csv(students,encoding="iso-8859-1")
school_df=pd.DataFrame(school_data)
student_df=pd.DataFrame(student_data)

school_df=school_df.rename(columns={"name":"school"})
school_df


,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [3]:
#set Boolean variables to asses passing

student_df["reading pass"]=student_df["reading_score"]>70
student_df["math pass"]=student_df["math_score"]>70
student_df["Passed Math & Reading"]=((student_df["reading pass"])&(student_df["math pass"]))

student_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score,reading pass,math pass,Passed Math & Reading
0,0,Paul Bradley,M,9th,Huang High School,66,79,False,True,False
1,1,Victor Smith,M,12th,Huang High School,94,61,True,False,False
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,True,False,False
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,False,False,False
4,4,Bonnie Ray,F,9th,Huang High School,97,84,True,True,True


In [4]:
#start calculating the various required summary information from the two dataframes
total_schools=len(school_df["school"])
total_students=len(student_df["name"])
total_budget=school_df["budget"].sum()
avg_math_score=student_df["math_score"].mean()
avg_read_score=student_df["reading_score"].mean()
math_pass=sum(student_df["math pass"])
math_pass_percent= (math_pass/total_students) * 100
read_pass=sum(student_df["reading pass"])
read_pass_percent=(read_pass/total_students) * 100
#This is not required, but I'm also reviewing the passing of both math and reading
total_pass=sum((student_df["reading pass"]) & (student_df["math pass"]))
total_pass_percent=(total_pass/total_students) * 100
overall_passing_rate = ((read_pass_percent + math_pass_percent)/2)


In [5]:
#take all of the summarized data and put it in a summary table, fomatted for the District
district_summary = pd.DataFrame({  "Total Schools": [total_schools],
                                   "Total Students": [total_students],
                                   "Total Budget": [total_budget],
                                   "Avg. Math Score": [avg_math_score],
                                   "Percent Passing Math": [math_pass_percent],
                                   "Avg Reading Score": [avg_read_score],
                                   "Percent Passing Reading":[read_pass_percent],
                                   "Overall Passing Rate":[overall_passing_rate] })
district_summary = district_summary[["Total Schools",
                                    "Total Students",
                                    "Total Budget",
                                    "Avg. Math Score",
                                    "Percent Passing Math",
                                    "Avg Reading Score",
                                    "Percent Passing Reading",
                                    "Overall Passing Rate"]]
district_summary=district_summary.round(2)
district_summary["Total Students"]=district_summary["Total Students"].map("{0:,d}".format)
district_summary["Percent Passing Math"]=district_summary["Percent Passing Math"].map("{0:,.2f}%".format)
district_summary["Percent Passing Reading"]=district_summary["Percent Passing Reading"].map("{0:,.2f}%".format)
district_summary["Overall Passing Rate"]=district_summary["Overall Passing Rate"].map("{0:,.2f}%".format)
district_summary["Total Budget"]=district_summary["Total Budget"].map("${0:,.0f}".format)
district_summary

,Total Schools,Total Students,Total Budget,Avg. Math Score,Percent Passing Math,Avg Reading Score,Percent Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,72.39%,81.88,82.97%,77.68%


In [6]:
#Now creating a Summary Table for all of the Schools
school_summary=school_df
school_summary=school_summary.rename(columns={"type":"School Type",
                                             "size":"Total Students",
                                             "budget":"Total School Budget"})
school_summary["Per Student Budget"]=school_summary["Total School Budget"]/school_summary["Total Students"]
school_summary.drop("School ID",axis=1,inplace=True)

first_gp=student_df.groupby(["school"])

In [7]:
#calculate the various variables required for the summary table
school_math=first_gp['math_score'].mean()
school_reading=first_gp['reading_score'].mean()
math_num=first_gp['math pass'].sum()
reading_num=first_gp['reading pass'].sum()
both_num=first_gp['Passed Math & Reading'].sum()
school_math_df=pd.DataFrame(school_math)
school_math_df["math pass num"]= math_num
school_math_df["Passed Both"]=both_num
school_math_df.reset_index(inplace=True)
school_reading_df=pd.DataFrame(school_reading)
school_reading_df["read pass num"]=reading_num
school_reading_df.reset_index(inplace=True)
math_and_read=pd.merge(school_math_df,school_reading_df,)
school_summary=pd.merge(school_summary,math_and_read,on="school")
school_summary["% Passing Math"]=school_summary["math pass num"]/school_summary["Total Students"]*100
school_summary["% Passing Reading"]=school_summary["read pass num"]/school_summary["Total Students"]*100
school_summary["Overall Passing Rate"]=((school_summary['% Passing Math']+school_summary['% Passing Reading'])/2)
school_summary["% Passed Both"]=school_summary['Passed Both']/school_summary["Total Students"]*100

In [8]:
#Additional Formatting for the table
final_school_summary=school_summary
final_school_summary=final_school_summary.rename(columns={"school":"School",
                                                         "math_score":"Avg. Math Score",
                                                         "reading_score":"Avg. Reading Score"})
final_school_summary.drop("math pass num",axis=1,inplace=True)
final_school_summary.drop("read pass num",axis=1,inplace=True)
final_school_summary=final_school_summary.round(2)
final_school_summary["Total Students"]=final_school_summary["Total Students"].map("{0:,d}".format)
final_school_summary["% Passing Math"]=final_school_summary["% Passing Math"].map("{0:,.2f}%".format)
final_school_summary["% Passing Reading"]=final_school_summary["% Passing Reading"].map("{0:,.2f}%".format)
final_school_summary["Overall Passing Rate"]=final_school_summary["Overall Passing Rate"].map("{0:,.2f}%".format)
final_school_summary["Total School Budget"]=final_school_summary["Total School Budget"].map("${0:,.0f}".format)
final_school_summary["Per Student Budget"]=final_school_summary["Per Student Budget"].map("${0:,.0f}".format)
final_school_summary["% Passed Both"]=final_school_summary["% Passed Both"].map("{0:,.2f}%".format)
#Summary for all schools
final_school_summary=final_school_summary.sort_values("School")
final_school_summary.reset_index(inplace=True)
final_school_summary

,index,School,School Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Passed Both,Avg. Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,% Passed Both
0,7,Bailey High School,District,"4,976","$3,124,928",$628,77.05,2545.0,81.03,64.63%,79.30%,71.97%,51.15%
1,6,Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,1561.0,83.98,89.56%,93.86%,91.71%,84.02%
2,1,Figueroa High School,District,"2,949","$1,884,411",$639,76.71,1472.0,81.16,63.75%,78.43%,71.09%,49.92%
3,13,Ford High School,District,"2,739","$1,763,916",$644,77.10,1405.0,80.75,65.75%,77.51%,71.63%,51.30%
4,4,Griffin High School,Charter,"1,468","$917,500",$625,83.35,1228.0,83.82,89.71%,93.39%,91.55%,83.65%
5,3,Hernandez High School,District,"4,635","$3,022,020",$652,77.29,2325.0,80.93,64.75%,78.19%,71.47%,50.16%
6,8,Holden High School,Charter,427,"$248,087",$581,83.80,359.0,83.81,90.63%,92.74%,91.69%,84.07%
7,0,Huang High School,District,"2,917","$1,910,635",$655,76.63,1456.0,81.18,63.32%,78.81%,71.07%,49.91%
8,12,Johnson High School,District,"4,761","$3,094,650",$650,77.07,2371.0,80.97,63.85%,78.28%,71.07%,49.80%
9,9,Pena High School,Charter,962,"$585,858",$609,83.84,816.0,84.04,91.68%,92.20%,91.94%,84.82%


In [9]:
#Formatted for the top 5 performing schools
final_school_summary=final_school_summary.sort_values("Overall Passing Rate",ascending=False)
final_school_summary.head(5)

,index,School,School Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Passed Both,Avg. Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,% Passed Both
13,5,Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,1938.0,83.99,90.93%,93.25%,92.09%,84.89%
9,9,Pena High School,Charter,962,"$585,858",$609,83.84,816.0,84.04,91.68%,92.20%,91.94%,84.82%
14,10,Wright High School,Charter,"1,800","$1,049,400",$583,83.68,1520.0,83.96,90.28%,93.44%,91.86%,84.44%
1,6,Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,1561.0,83.98,89.56%,93.86%,91.71%,84.02%
6,8,Holden High School,Charter,427,"$248,087",$581,83.80,359.0,83.81,90.63%,92.74%,91.69%,84.07%


In [10]:
#formatted for the 5 worst performing schools
final_school_summary=final_school_summary.sort_values("Overall Passing Rate",ascending=True)
final_school_summary.head(5)

,index,School,School Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Passed Both,Avg. Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,% Passed Both
10,11,Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,1977.0,80.74,64.07%,77.74%,70.91%,49.44%
7,0,Huang High School,District,"2,917","$1,910,635",$655,76.63,1456.0,81.18,63.32%,78.81%,71.07%,49.91%
8,12,Johnson High School,District,"4,761","$3,094,650",$650,77.07,2371.0,80.97,63.85%,78.28%,71.07%,49.80%
2,1,Figueroa High School,District,"2,949","$1,884,411",$639,76.71,1472.0,81.16,63.75%,78.43%,71.09%,49.92%
5,3,Hernandez High School,District,"4,635","$3,022,020",$652,77.29,2325.0,80.93,64.75%,78.19%,71.47%,50.16%


In [11]:
#Table to look at the Math Scores across all schools and by Class
by_grade=student_df
by_grade['grade']=by_grade['grade'].astype('category', categories=['9th','10th','11th','12th'])
math_scores_grade= round(by_grade.pivot_table(index="school",columns="grade",values="math_score"),2)
math_scores_grade


grade,9th,10th,11th,12th
school,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [12]:
#Table to look at Reading scores across all schools by grade
reading_scores_grade= round(by_grade.pivot_table(index="school",columns="grade",values="reading_score"),2)
reading_scores_grade

grade,9th,10th,11th,12th
school,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [13]:
#binning on spending to look at schools grouped by the average spend per student
spending_bin=[0,599,625,640,1000]
spending_labels=[" < $600 ", "$600 - $625", "$626 - $640"," > $640 "]
binned_spend=school_summary
binned_spend["Spending Ranges per Student"]=pd.cut(binned_spend["Per Student Budget"],spending_bin,labels=spending_labels)
spend_grouped_schools=pd.pivot_table(binned_spend, index=["Spending Ranges per Student"])
per_student_spend=round(spend_grouped_schools[['math_score','reading_score','% Passing Math','% Passing Reading','Overall Passing Rate','% Passed Both']],2)
per_student_spend=per_student_spend.rename(columns={"math_score":"Avg. Math Score","reading_score":"Avg. Reading Score"})

per_student_spend

,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,% Passed Both
Spending Ranges per Student,,,,,,
< $600,83.46,83.93,90.35,93.33,91.84,84.36
$600 - $625,83.52,83.86,90.43,92.74,91.58,83.89
$626 - $640,78.51,81.70,70.67,82.10,76.38,58.69
> $640,77.02,80.96,64.42,78.20,71.31,50.29


In [14]:
#binning on school size to look at schools grouped by the average size of schools
size_bin=[0,1000,2000,4000,8000]
size_labels=[" < 1,000 ", "1,000 - 2,000", "2,000 - 4,000"," > 4,000"]
binned_size=school_summary
binned_size["Number of Students"]=pd.cut(binned_size["Total Students"],size_bin,labels=size_labels)
size_grouped_schools=pd.pivot_table(binned_size, index=["Number of Students"])
per_student_size=round(size_grouped_schools[['math_score','reading_score','% Passing Math','% Passing Reading','Overall Passing Rate','% Passed Both']],2)
per_student_size=per_student_size.rename(columns={"math_score":"Avg. Math Score","reading_score":"Avg. Reading Score"})

per_student_size

,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,% Passed Both
Number of Students,,,,,,
"< 1,000",83.82,83.93,91.16,92.47,91.82,84.45
"1,000 - 2,000",83.37,83.86,89.93,93.24,91.59,83.92
"2,000 - 4,000",78.11,81.56,69.56,81.15,75.36,57.09
"> 4,000",77.14,80.98,64.41,78.59,71.50,50.37


In [15]:
#Table to look at the two school types and their scores
type_of_school=pd.pivot_table(school_summary,index=["School Type"])
type_of_school=round(type_of_school[['math_score','reading_score','% Passing Math','% Passing Reading','Overall Passing Rate','% Passed Both']],2)
type_of_school=type_of_school.rename(columns={"math_score":"Avg. Math Score","reading_score":"Avg. Reading Score"})
type_of_school

,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,% Passed Both
School Type,,,,,,
Charter,83.47,83.90,90.36,93.05,91.71,84.17
District,76.96,80.97,64.30,78.32,71.31,50.24
